In [7]:
import pandas as pd
import json

class DataProcessor:
    def __init__(self):
        self.city_mapping_corrected = {
            'İLKADIM/SAMSUN': 'Samsun/Ilkadim',
            'BEŞIKTAŞ/İSTANBUL': 'Istanbul/Beşiktaş',
            'BAYRAMPAŞA/İSTANBUL': 'Istanbul/Bayrampaşa',
            'ÜMRANİYE/İSTANBUL': 'Istanbul/Ümraniye',
            'SOMA/MANİSA': 'Manisa/Soma',
            'BUCA/İZMİR': 'Izmir/Buca',
            'ORTAKÖY/AKSARAY': 'Aksaray/Ortaköy',
            'BORNOVA/İZMİR': 'Izmir/Bornova',
            'KEPEZ/ANTALYA': 'Antalya/Kepez',
            'BAHÇELİEVLER/İSTANBUL': 'Istanbul/Bahçelievler',
            'KARTAL/İSTANBUL': 'Istanbul/Kartal',
            'OSMANGAZİ/BURSA': 'Bursa/Osmangazi',
            'KAĞITHANE/İSTANBUL': 'Istanbul/Kağithane',
            'FATIH/İSTANBUL.': 'Istanbul/Fatih',
            'GÜNGÖREN/İSTANBUL': 'Istanbul/Güngören',
            'FATİH/İSTANBUL': 'Istanbul/Fatih',
            'DERİNCE/KOCAELİ': 'Kocaeli/Derince',
            'KÜÇÜKÇEKMECE/İSTANBUL': 'Istanbul/Küçükçekmece',
            'CANİK/SAMSUN': 'Samsun/Canik',
            'ELMADAĞ/ANKARA': 'Ankara/Elmadağ',
            'ATAŞEHİR/İSTANBUL': 'Istanbul/Ataşehir',
            'FATIH/İSTANBUL': 'Istanbul/Fatih',
            'SANCAKTEPE/İSTANBUL': 'Istanbul/Sancaktepe',
            'ŞİŞLİ/İSTANBUL': 'Istanbul/Şişli',
            'ODUNPAZARI/ESKİŞEHİR': 'Eskişehir/Odunpazarı',
            'ESENYURT/İSTANBUL': 'Istanbul/Esenyurt',
            'BULANCAK/GİRESUN': 'Giresun/Bulancak',
            'ALTINDAĞ/ANKARA': 'Ankara/Altındağ',
            'ŞEHİTKAMİL/GAZİANTEP': 'Gaziantep/Şehitkamil'
        }
        self.socioeconomic_data = None
        self.load_socioeconomic_data('data/formatted_data.json')
        
    def read_csv(self, file_path, remove_columns=None):
        df = pd.read_csv(file_path, low_memory=False)
        if remove_columns:
            df.drop(remove_columns, axis=1, inplace=True)
        return df
    
    def map_column(self, df, column_name, mapping_dict):
        df[column_name] = df[column_name].map(mapping_dict).fillna(df[column_name])
        return df
    
    def group_and_aggregate(self, df, group_by_column, agg_dict):
        return df.groupby(group_by_column).agg(agg_dict).reset_index()

    def merge(self, df1, df2, key, how='left'):
        return pd.merge(df1, df2, on=key, how=how)

    def load_socioeconomic_data(self, file_path):
        with open(file_path, 'r') as f:
            self.socioeconomic_data = json.load(f)

    def add_socioeconomic_info(self, df, city_column):
        df['SocioeconomicData'] = df[city_column].apply(lambda x: self.socioeconomic_data.get(x, None))
        return df
    
    def rename_columns(self, df, new_column_names):
        df.rename(columns=new_column_names, inplace=True)
        return df

    def process_data(self):
        df = self.read_csv('data/new_dataframe.csv', remove_columns=['Unnamed: 0'])
        rfm = self.read_csv('data/Customer_rfm.csv')
        churn_df = self.read_csv("data/churn_analysis.csv")
        
        df = self.map_column(df, 'Sehir', self.city_mapping_corrected)
        
        new_df = df[['id', 'Sehir', 'Tercih']].copy()
        new_df_grouped = self.group_and_aggregate(new_df, 'id', {
            'Sehir': 'first',  
            'Tercih': lambda x: list(set(x))
        })
        new_df_grouped['Tercih'] = new_df_grouped['Tercih'].apply(lambda x: x[0] if len(x) == 1 else x)
        
        merged_df = self.merge(rfm, new_df_grouped, 'id')
        result_df = self.merge(churn_df, merged_df, 'id', how='outer')
        
        result_df = self.add_socioeconomic_info(result_df, 'Sehir')
        
        column_names_mapping = {
            'id': 'CustomerID',
            'Churn': 'ChurnStatus',
            'Recency': 'RecencyScore',
            'Frequency': 'FrequencyScore',
            'Monetary': 'MonetaryValue',
            'R': 'RecencyRank',
            'F': 'FrequencyRank',
            'M': 'MonetaryRank',
            'RFM_Segment': 'RFMSegment',
            'RFM_Score': 'RFMScore',
            'Status': 'Segment',
            'Description': 'SegmentDescription',
            'Sehir': 'City',
            'Tercih': 'PreferredSolution',
            'socioeconomic_info': 'SocioeconomicData'
        }
        result_df = self.rename_columns(result_df, column_names_mapping)
        
        
        return result_df[['CustomerID', 'FirstTransactionDate', 'LastTransactionDate',
       'DaysSinceLastTransaction', 'ChurnStatus', 'TransactionFrequency', 'RecencyRank',
       'FrequencyRank', 'MonetaryRank', 'RFMScore','Segment', 'SegmentDescription', 'City', 'PreferredSolution',
       'SocioeconomicData']]

In [8]:
processor = DataProcessor()
final_df = processor.process_data()
final_df

,CustomerID,FirstTransactionDate,LastTransactionDate,DaysSinceLastTransaction,ChurnStatus,TransactionFrequency,RecencyRank,FrequencyRank,MonetaryRank,RFMScore,Segment,SegmentDescription,City,PreferredSolution,SocioeconomicData
0,301002470,2015-09-18 14:35:04,2023-10-26 10:47:00,0,False,442.714286,3,3,4,10,Platinum,Yüksek değerli müşteri. Ortalamanın çok üzerin...,Samsun/Ilkadim,"[ÖKC, CeptePos]","{'YASLI/GENC ORANI': {'Genc': 50.0, 'Orta': 21..."
1,301002583,2014-09-25 10:08:52,2020-12-23 12:28:28,1037,True,9.533333,1,1,1,3,Bronz,Düşük değerli müşteri. Ortalamanın çok altında...,Istanbul/Beşiktaş,CeptePos,"{'YAŞLI/GENÇ ORANI': {'Genç': 19.0, 'Orta': 22..."
2,301003354,2022-11-21 18:05:00,2023-10-26 16:59:00,0,False,2551.333333,3,4,3,10,Platinum,Yüksek değerli müşteri. Ortalamanın çok üzerin...,Istanbul/Bayrampaşa,ÖKC,"{'Yaşlı/Genç Oranı': {'Genç': 65.0, 'Orta': 4...."
3,301006906,2016-01-29 11:00:43,2022-09-12 18:30:00,408,True,618.350649,1,4,3,8,Altın,Orta değerli müşteri. Ortalamanın üzerinde sık...,Istanbul/Ümraniye,"[ÖKC, CeptePos]","{'YAŞLI/GENÇ ORANI': {'Genc': 95.5, 'Orta': 35..."
4,301009366,2019-06-27 17:31:14,2022-09-19 18:01:00,401,True,478.225000,2,3,2,7,Gümüş,Düşük-orta değerli müşteri. Ortalamanın altınd...,Manisa/Soma,"[ÖKC, CeptePos]","{'YAŞLI/GENÇ ORANI': {'Genç': 16.0, 'Orta': 31..."
5,301009412,2016-08-26 11:00:35,2023-10-03 17:56:28,22,False,1.333333,3,1,1,5,Gümüş,Düşük-orta değerli müşteri. Ortalamanın altınd...,Istanbul/Beşiktaş,CeptePos,"{'YAŞLI/GENÇ ORANI': {'Genç': 19.0, 'Orta': 22..."
6,301009682,2016-05-19 10:29:01,2023-10-26 16:39:00,0,False,176.011765,3,1,4,8,Altın,Orta değerli müşteri. Ortalamanın üzerinde sık...,Izmir/Buca,"[ÖKC, CeptePos]","{'Yaşlı/Genç Oranı': {'Genç': 20.0, 'Orta': 53..."
7,301010389,2016-06-15 15:12:25,2020-05-10 11:09:14,1264,True,510.424242,1,2,1,4,Bronz,Düşük değerli müşteri. Ortalamanın çok altında...,Aksaray/Ortaköy,CeptePos,"{'YAŞLI/GENÇ ORANI': {'Genç': 949.0, 'Orta': 9..."
8,301012873,2018-12-07 12:52:00,2023-10-26 15:06:00,0,False,616.800000,3,3,2,8,Altın,Orta değerli müşteri. Ortalamanın üzerinde sık...,Izmir/Bornova,ÖKC,"{'Yaşlı/Genç Oranı': {'Genç': 50.0, 'Orta': 36..."
9,301014561,2017-08-10 14:22:03,2023-10-26 16:55:00,0,False,1061.166667,3,3,3,9,Altın,Orta değerli müşteri. Ortalamanın üzerinde sık...,Antalya/Kepez,"[ÖKC, CeptePos]","{'Yaşlı/Genç Oranı': {'Genç': 68.0, 'Orta': 25..."


In [4]:
final_df.to_csv("data/odeal_customer_data_enhancement.csv")

## 📑 Özelliklerin Detaylı Açıklaması

- `CustomerID`: Her müşteriye özel belirlenen unique customer Id. 🆔
- `FirstTransactionDate`: Müşterinin gerçekleştirdiği ilk işlemin tarihi. 🌟
- `LastTransactionDate`: Müşterinin gerçekleştirdiği son işlemin tarihi. 🏷️
- `DaysSinceLastTransaction`: En son işlemden itibaren geçen gün sayısı. 🗓️
- `ChurnStatus`: müşterinin belirli bir süre içerisinde şirketle etkileşime geçip geçmediğini gösterir. Eğer müşteri, belirlenen 90 günlük süre zarfında şirketten herhangi bir hizmet almamış ve/veya işlem yapmamışsa, bu durum müşterinin şirketi terk etmiş olabileceğini işaret eder ve bu durumda ChurnStatus değeri True olarak belirlenir. Aksine, müşteri aktifse ve bu süre içerisinde işlem yapmışsa ChurnStatus değeri False olur, yani müşterinin hala aktif olduğunu ve şirketle ilişkisinin devam ettiğini gösterir. ✅❌
- `TransactionFrequency`: Belirli bir dönemdeki işlem sayısı. 🔄
- `RecencyRank`: Müşterinin en son işlemine göre sıralaması. 🆕
- `FrequencyRank`: İşlem sıklığına göre müşterinin sıralaması. 📊
- `MonetaryRank`: Toplam harcama tutarına göre müşterinin sıralaması. 💸
- `RFMScore`: Yenilik/Güncellik (Recency), Sıklık (Frequency) ve Parasal (Monetary) değerlerin toplam puanı. 🔢
- `Segment`: Müşterinin ait olduğu segment. 🎚️
  - `Altın`: Çok yüksek değerli müşteriler, sık işlem yaparlar ve yüksek meblağlar harcarlar. 🥇
  - `Gümüş`: Yüksek değerli müşteriler, sıklıkla işlem yaparlar ve orta düzeyde harcama yaparlar. 🥈
  - `Platinum`: Orta seviye değerli müşteriler, ortalama sıklıkta işlem yaparlar ve ortalama harcama yaparlar. 🥉
  - `Bronz`: Daha düşük değerli müşteriler, daha az sıklıkta işlem yaparlar ve daha az harcama yaparlar. 🏅
- `SegmentDescription`: Müşteri segmentinin tanımı. 🔍
- `City`: Müşterinin hizmet verdiği şehir. 🏙️
- `PreferredSolution`: Müşterinin tercih ettiği hizmet/ler ya da ürün tipi. 💡
- `SocioeconomicData`: Müşterinin hizmet verdiği şehir ile ilgili sosyoekonomik veriler. 📈

Sonuç olarak oluşan yeni dataframe, müşteri sadakatini ve yaşam boyu değerini artırmak için stratejik planlamalar yapılmasına olanak tanır.

---

